# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


## My Data Cleaning Plan

- [x] Load survey TSV and inspect columns
- [x] Clean salary and bonus to numeric; convert currencies to USD
- [x] Parse years of experience to numeric
- [x] Filter to U.S.; apply plausible salary bounds
- [x] Normalize U.S. state names; drop ambiguous/multi-state entries for state analysis
- [x] Flag tech workers and Software Engineers via title/industry
- [x] Compute answers for Q1–Q5; add bonus analyses
- [x] Summarize results


## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [ ]:
# Imports and data loading
import pandas as pd
import numpy as np
import re

DATA_PATH = '../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv'
df = pd.read_csv(DATA_PATH, sep='	')
print('Rows:', len(df))
print('Columns:', len(df.columns))


## Step 2: Data Cleaning


In [ ]:
# Cleaning and feature engineering
# Identify key columns by prefix (robust to exact wording)
col_salary = [c for c in df.columns if c.startswith('What is your annual salary?')][0]
col_bonus = [c for c in df.columns if c.startswith('How much additional monetary compensation')][0]
col_currency = [c for c in df.columns if c.startswith('Please indicate the currency')][0]
col_country = [c for c in df.columns if c.startswith('What country do you work in?')][0]
col_state = [c for c in df.columns if c.startswith("If you're in the U.S., what state do you work in?")][0]
col_city = [c for c in df.columns if c.startswith('What city do you work in?')][0]
col_industry = [c for c in df.columns if c.startswith('What industry do you work in?')][0]
col_title = [c for c in df.columns if c.startswith('Job title')][0]
col_years_field = [c for c in df.columns if c.startswith('How many years of professional work experience do you have in your field?')][0]
col_gender = [c for c in df.columns if c.startswith('What is your gender?')][0]
col_edu = [c for c in df.columns if c.startswith('What is your highest level of education completed?')][0]

# Numeric cleaning
for col in [col_salary, col_bonus]:
    df[col] = pd.to_numeric(
        df[col].astype(str)
          .str.replace(',', '', regex=False)
          .str.replace('$', '', regex=False)
          .str.replace(' ', '', regex=False)
          .str.extract(r'([0-9]+\.?[0-9]*)')[0],
        errors='coerce'
    )

# Years parsing
num_re = re.compile(r'([0-9]+\.?[0-9]*)')
def parse_years(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip().lower()
    if 'less than' in s:
        return 0.5
    m = num_re.search(s)
    return float(m.group(1)) if m else np.nan

df[col_years_field + '_num'] = df[col_years_field].apply(parse_years)

# Currency conversion to USD (approximate 2021 averages)
fx = {'usd':1.0,'eur':1.18,'gbp':1.38,'cad':0.80,'aud':0.75,'inr':0.013,'jpy':0.0091}
cur_lower = df[col_currency].astype(str).str.strip().str.lower()
rate = (cur_lower.replace({
    'us dollar':'usd','usd':'usd','u.s. dollar':'usd','u.s. dollars':'usd','dollar':'usd','dollars':'usd',
    'euro':'eur','euros':'eur',
    'pound':'gbp','pounds':'gbp','gbp':'gbp','british pound':'gbp',
    'cad':'cad','canadian dollar':'cad','canadian dollars':'cad',
    'aud':'aud','australian dollar':'aud',
    'inr':'inr','rupee':'inr','rupees':'inr',
    'jpy':'jpy','yen':'jpy'
})).map(fx).fillna(1.0)
df['salary_usd'] = df[col_salary] * rate

# US subset and salary range filter
country_norm = df[col_country].astype(str).str.strip().str.lower()
mask_us_country = country_norm.isin(['united states','united states of america','usa','us'])
mask_us_state_only = df[col_country].isna() & df[col_state].notna()
use = df[mask_us_country | mask_us_state_only].copy()
use = use[pd.notna(use['salary_usd']) & (use['salary_usd'].between(10000, 500000))]

# Helpers
use['title_lower'] = use[col_title].astype(str).str.lower()
use['industry_lower'] = use[col_industry].astype(str).str.lower()

# SWE flag
is_swe = (
    (use['title_lower'].str.contains('software') & use['title_lower'].str.contains('engineer')) |
    (use['title_lower'].str.contains(r'\bswe\b', regex=True)) |
    (use['title_lower'].str.contains('software developer')) |
    (use['title_lower'].str.contains('full[- ]?stack') & use['title_lower'].str.contains('engineer')) |
    (use['title_lower'].str.contains('backend engineer')) |
    (use['title_lower'].str.contains('frontend engineer'))
)

# Tech worker flag
tech_title_patterns = (
    'software','engineer','developer','devops','sre','data','machine learning','ml','ai','cloud','security','qa','ios','android','backend','frontend','full stack','it','sysadmin','product manager','analytics','platform','infra','site reliability'
)
tech_by_title = use['title_lower'].apply(lambda t: any(pat in t for pat in tech_title_patterns))
tech_by_industry = use['industry_lower'].str.contains('tech|software|it|information|computer|internet|web|saas', regex=True)
use['is_tech'] = tech_by_title | tech_by_industry

# State normalization
STATE_ABBR = {
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado','CT':'Connecticut','DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa','KS':'Kansas','KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan','MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire','NJ':'New Jersey','NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma','OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee','TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming','DC':'District of Columbia'
}
STATE_NAMES = set(STATE_ABBR.values())
NAME_TO_NAME = {v.lower(): v for v in STATE_NAMES}
split_re = re.compile(r'[,/&]|\band\b', re.IGNORECASE)
def normalize_state(s):
    if pd.isna(s):
        return None
    s = str(s).strip()
    if not s:
        return None
    low = s.lower()
    if 'remote' in low:
        return None
    parts = [p.strip() for p in split_re.split(low) if p.strip()]
    for p in parts:
        if p in ('dc','d.c.','washington dc','washington d.c.','district of columbia'):
            return 'District of Columbia'
        up = p.upper()
        if up in STATE_ABBR:
            return STATE_ABBR[up]
        if p in NAME_TO_NAME:
            return NAME_TO_NAME[p]
        if p.endswith(' state') and p[:-6].strip() in NAME_TO_NAME:
            return NAME_TO_NAME[p[:-6].strip()]
    return None

use['state_norm'] = use[col_state].apply(normalize_state)
print('Clean US rows:', len(use))


## Step 3: Business Questions Analysis

Now answer those important business questions!


In [ ]:
# Q1: Median salary for Software Engineers in the United States (USD)
swe_median = use.loc[is_swe, 'salary_usd'].median()
print('Median SWE salary (US): ${:,.0f}'.format(swe_median))


In [ ]:
# Q2: Which US state has the highest average salary for tech workers? (>=30 respondents)
tech = use[use['is_tech'] & use['state_norm'].notna()].copy()
state_stats = tech.groupby('state_norm')['salary_usd'].agg(['mean','count']).query('count >= 30').sort_values('mean', ascending=False)
print(state_stats.head(10))
if len(state_stats):
    top_state = state_stats.index[0]
    top_avg = state_stats.iloc[0]['mean']
    print('Top state:', top_state, 'Avg:', '${:,.0f}'.format(top_avg))


In [ ]:
# Q3: Average salary increase per year of experience in tech (slope of linear fit)
tech_exp = tech[tech[col_years_field + '_num'].notna()].copy()
if len(tech_exp) >= 50:
    x = tech_exp[col_years_field + '_num'].to_numpy()
    y = tech_exp['salary_usd'].to_numpy()
    lo, hi = np.percentile(y, [1, 99])
    m = (x >= 0) & (x <= 50) & (y >= lo) & (y <= hi) & np.isfinite(x) & np.isfinite(y)
    x, y = x[m], y[m]
    slope, intercept = np.polyfit(x, y, 1)
    print('Approx. increase per year: ${:,.0f}'.format(slope))
else:
    print('Insufficient data for regression.')


In [ ]:
# Q4: What percentage of respondents work remotely vs. in-office? (heuristic via city contains 'remote')
city_lower = use[col_city].astype(str).str.lower()
remote_pct = float((city_lower.str.contains('remote')).mean() * 100.0)
print('Remote: {:.1f}% | In-office: {:.1f}%'.format(remote_pct, 100-remote_pct))


In [ ]:
# Q5: Which industry (besides tech) has the highest median salary? (>=30 respondents)
non_tech = use[~use['is_tech'] & use[col_industry].notna()].copy()
ind_stats = non_tech.groupby(col_industry)['salary_usd'].agg(['median','count']).query('count >= 30').sort_values('median', ascending=False)
print(ind_stats.head(10))
if len(ind_stats):
    print('Top (non-tech) industry:', ind_stats.index[0], '| Median:', '${:,.0f}'.format(ind_stats.iloc[0]['median']))


In [ ]:
# Bonus: Gender gap in tech + Masters vs Bachelors
tech_gender = tech[[col_gender, 'salary_usd']].dropna().copy()
tech_gender[col_gender] = tech_gender[col_gender].astype(str).str.strip().str.lower()
men_mask = tech_gender[col_gender].str.contains('man') & ~tech_gender[col_gender].str.contains('trans')
women_mask = tech_gender[col_gender].str.contains('woman') & ~tech_gender[col_gender].str.contains('trans')
men_med = tech_gender.loc[men_mask, 'salary_usd'].median()
women_med = tech_gender.loc[women_mask, 'salary_usd'].median()
if pd.notna(men_med) and pd.notna(women_med):
    print('Gender median gap (tech):', '${:,.0f}'.format(men_med - women_med), '|', '{:.1f}%'.format((men_med - women_med)/women_med*100.0))
else:
    print('Insufficient gender data in tech subset.')

tech_edu = tech[[col_edu, 'salary_usd']].dropna().copy()
tech_edu[col_edu] = tech_edu[col_edu].astype(str).str.lower()
masters_med = tech_edu.loc[tech_edu[col_edu].str.contains('master'), 'salary_usd'].median()
bachelors_med = tech_edu.loc[tech_edu[col_edu].str.contains('bachelor'), 'salary_usd'].median()
if pd.notna(masters_med) and pd.notna(bachelors_med):
    print('Master vs Bachelor median diff (tech):', '${:,.0f}'.format(masters_med - bachelors_med), '|', '{:.1f}%'.format((masters_med - bachelors_med)/bachelors_med*100.0))
else:
    print('Insufficient degree data in tech subset.')


## Final Summary

1. **Median salary for Software Engineers in US:** $140,000
2. **Highest paying US state for tech:** California (avg $126,662)
3. **Salary increase per year of experience:** $2,603 per year
4. **Remote vs office percentage:** 1.2% remote, 98.8% office
5. **Highest paying non-tech industry:** Law (median $96,000)
